In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd

df = pd.DataFrame()
boba = 'boba'
df.to_csv(f'results/{boba}.csv', index=False)

In [2]:
import os

weights_dir = "/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/checkpoints/4-class-resnet-100-epochs"
weights = os.listdir(weights_dir)

for w in weights:
    if w[-4:] == '.pth':
        print(weights_dir + '/' + w)

/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/checkpoints/4-class-resnet-100-epochs/model_epoch_latest.pth
/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/checkpoints/4-class-resnet-100-epochs/model_epoch_0.pth
/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/checkpoints/4-class-resnet-100-epochs/model_epoch_20.pth
/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/checkpoints/4-class-resnet-100-epochs/model_epoch_40.pth
/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/checkpoints/4-class-resnet-100-epochs/model_epoch_60.pth
/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/checkpoints/4-class-resnet-100-epochs/model_epoch_80.pth
/home/jovyan/shares/SR006.nfs2/

In [2]:
import sys
import time
import os
import csv
import torch
from util import Logger, printSet
from validate import validate
from networks.resnet import resnet50
from options.test_options import TestOptions
import networks.resnet as resnet
import numpy as np
import random
import pandas as pd


def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
seed_torch(100)
DetectionTests = {
                'ForenSynths': { 'dataroot'   : '../../datasets/ForenSynths/',
                                 'no_resize'  : False, # Due to the different shapes of images in the dataset, resizing is required during batch detection.
                                 'no_crop'    : True,
                               },

           'GANGen-Detection': { 'dataroot'   : '../../datasets/GANGen-Detection/',
                                 'no_resize'  : True,
                                 'no_crop'    : True,
                               },

         'DiffusionForensics': { 'dataroot'   : '../../datasets/DiffusionForensics/',
                                 'no_resize'  : False, # Due to the different shapes of images in the dataset, resizing is required during batch detection.
                                 'no_crop'    : True,
                               },

        'UniversalFakeDetect': { 'dataroot'   : '../../datasets/UniversalFakeDetect/',
                                 'no_resize'  : False, # Due to the different shapes of images in the dataset, resizing is required during batch detection.
                                 'no_crop'    : True,
                               },

        'Diffusion1kStep': { 'dataroot'   : '../../datasets/Diffusion1kStep/',
                                 'no_resize'  : False, # Due to the different shapes of images in the dataset, resizing is required during batch detection.
                                 'no_crop'    : True,
                               },

                 }


weights_dir = "/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/checkpoints/4-class-resnet-100-epochs"
weights = os.listdir(weights_dir)

for w in weights:
    if w[-4:] == '.pth':
      opt = TestOptions().parse(print_options=False)

      # params
      opt.model_path = weights_dir + '/' + w
      opt.batch_size = 1024
      print(f'Model_path {opt.model_path}')
      torch.cuda.set_device('cuda:0')

      # get model
      model = resnet50(num_classes=1)
      model.load_state_dict(torch.load(opt.model_path, map_location='cuda:0'), strict=True)
      model.cuda()
      model.eval()

      results = []

      for testSet in DetectionTests.keys():
          dataroot = DetectionTests[testSet]['dataroot']
          printSet(testSet)

          accs = []
          aps = []
          print(time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime()))
          for v_id, val in enumerate(os.listdir(dataroot)):
              opt.dataroot = '{}/{}'.format(dataroot, val)
              opt.classes  = '' #os.listdir(opt.dataroot) if multiclass[v_id] else ['']
              opt.no_resize = DetectionTests[testSet]['no_resize']
              opt.no_crop   = DetectionTests[testSet]['no_crop']
              acc, ap, _, _, _, _ = validate(model, opt)
              accs.append(acc)
              aps.append(ap)
              print("({} {:12}) acc: {:.1f}; ap: {:.1f}".format(v_id, val, acc*100, ap*100))
              results.append({'group': testSet, 'id': v_id, 'dataset': val, 'accuracy': acc * 100, 'average_precision': ap * 100})

          results.append({'group': testSet, 'id': v_id + 1, 'dataset': 'Mean', 'accuracy': np.array(accs).mean() * 100, 'average_precision': np.array(aps).mean() * 100})    #TODO: изменить dataframe
          print("({} {:10}) acc: {:.1f}; ap: {:.1f}".format(v_id+1,'Mean', np.array(accs).mean()*100, np.array(aps).mean()*100))
          print('*'*25) 

      df = pd.DataFrame(results)
      df.to_csv(f'results/{w[:-4]}.csv', index=False)

Model_path ./checkpoints/test-4class-resnet-car-cat-chair-horse/model_epoch_last.pth


/tmp/ipykernel_692651/172713978.py:67: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(opt.model_path, map_location='cuda:0'), strict=True)


           ForenSynths
2024_08_08_01_28_13
(0 biggan      ) acc: 80.5; ap: 90.6
(1 crn         ) acc: 50.0; ap: 51.7
(2 cyclegan    ) acc: 79.0; ap: 99.2
(3 deepfake    ) acc: 58.6; ap: 67.3
(4 gaugan      ) acc: 83.5; ap: 85.7
(5 imle        ) acc: 50.0; ap: 51.7
(6 progan      ) acc: 99.8; ap: 100.0
(7 san         ) acc: 72.1; ap: 74.8
(8 seeingdark  ) acc: 68.1; ap: 61.2
(9 stargan     ) acc: 89.3; ap: 100.0
(10 stylegan    ) acc: 91.9; ap: 99.9
(11 stylegan2   ) acc: 93.2; ap: 100.0
(12 whichfaceisreal) acc: 77.7; ap: 75.9
(13 Mean      ) acc: 76.4; ap: 81.4
*************************
                GANGen-Detection
2024_08_08_01_40_24
(0 AttGAN      ) acc: 69.9; ap: 97.6
(1 BEGAN       ) acc: 100.0; ap: 100.0
(2 MMDGAN      ) acc: 98.8; ap: 99.9
(3 RelGAN      ) acc: 99.7; ap: 100.0
(4 S3GAN       ) acc: 71.9; ap: 80.4
(5 SNGAN       ) acc: 83.2; ap: 98.5
(6 STGAN       ) acc: 84.2; ap: 100.0
(7 CramerGAN   ) acc: 99.2; ap: 100.0
(8 InfoMaxGAN  ) acc: 80.3; ap: 98.0
(9 Mean      )